In [6]:
import requests
import pandas as pd
import concurrent.futures
import json
import os

In [7]:
league_id = '366970'
general_endpoint = "https://fantasy.premierleague.com/api/bootstrap-static/"
league_endpoint = f'https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings'
fixture_endpoint = 'https://fantasy.premierleague.com/api/fixtures/'

In [8]:
def fetch_data(general_endpoint, league_endpoint, fixture_endpoint):
    with requests.Session() as session:
        general_response = session.get(general_endpoint).json()
        league_response = session.get(league_endpoint).json()
        fixture_response = session.get(fixture_endpoint).json()

    real_player_data = pd.DataFrame.from_records(general_response['elements'], columns=['id', 'web_name', 'selected_by_percent', 'total_points', 'bonus', 'form', 'value_season', 'value_form' ]).astype({"form": float, "value_season": float, "value_form": float}).set_index('id')
    gameweek_data = pd.DataFrame.from_records(general_response['events'], columns=['id', "deadline_time",'is_current', 'most_captained', 'most_vice_captained', 'top_element']).fillna(0).astype({"most_captained": int, "most_vice_captained": int, "top_element": int}).set_index('id')
    real_team_names = pd.DataFrame.from_records(general_response['teams'], columns=['id', 'name'])
    fixture_ids = pd.DataFrame.from_records(fixture_response, columns =['event', 'team_a', 'team_h'])
    current_standings = pd.DataFrame.from_records(league_response['standings']['results'], columns=['id', 'entry', 'player_name', 'rank', 'last_rank', 'total']).astype({"rank": int, "last_rank": int, "total": int}).set_index('entry')
    current_week = gameweek_data.index[gameweek_data['is_current']].item()
    manager_teams = pd.concat([pd.DataFrame.from_records(session.get(f'https://fantasy.premierleague.com/api/entry/{i}/event/{current_week}/picks/').json()['picks'], ).assign(entry=i) for i in current_standings.index])
    current_gameweek_teams = pd.merge(manager_teams, real_player_data, left_on='element', right_index=True)

    template_teams = current_gameweek_teams.astype({"selected_by_percent": float}).groupby('entry')['selected_by_percent'].mean().sort_values(ascending=False).to_frame(name='selected_by_percent')
    template_teams = current_standings.join(template_teams, on='entry')[['player_name', 'rank', 'total', 'selected_by_percent']]


    # JSON string with team names and ids
    json_str = real_team_names.to_json(orient='records')

    # Convert JSON string to dictionary
    team_dict = json.loads(json_str)

    # Create a dictionary of team IDs and names
    team_id_name_dict = {team['id']: team['name'] for team in team_dict}

    fixture_ids['team_a_name'] = fixture_ids['team_a'].map(team_id_name_dict)
    fixture_ids['team_h_name'] = fixture_ids['team_h'].map(team_id_name_dict)

    fixture_teams_file_path = 'fixture_teams.csv'
    if not os.path.isfile(fixture_teams_file_path):
        permanent_fixture_team = current_standings
        permanent_fixture_team['fixture_team'] = permanent_fixture_team['rank'].map(team_id_name_dict)
        permanent_fixture_team = current_standings.iloc[:, [1,5]]
        permanent_fixture_team.to_csv(fixture_teams_file_path, index=False)
    else:
        permanent_fixture_team = pd.read_csv(fixture_teams_file_path)


    return gameweek_data, current_standings, current_gameweek_teams, template_teams, fixture_ids, permanent_fixture_team

In [9]:
gameweek_data, current_standings, current_gameweek_teams, template_teams, fixture_ids, permanent_fixture_team = fetch_data(general_endpoint, league_endpoint,fixture_endpoint)

In [10]:
permanent_fixture_team

,player_name,fixture_team
0,Divyam Dixit,pap
1,Marco Gouveia,Aston Villa
2,Connor McDonald,Bournemouth
3,Alex Wietzorrek,Brentford
4,Ryan Shacks,Brighton
5,Peter Wertz,Chelsea
6,Niklas Wietzorrek,Crystal Palace
7,Devon Hodgson,Everton
8,Devon Jansen,Fulham
9,Liam Smorfitt,Leicester
